#Credit Scoring Model

On this occasion, I’m about to build a credit scoring model using two classification algorithms: Linear Regression and Decision Tree.

- Linear regression is used because it is based on binary logit, the variable is a dummy variable that uses two possible values, 0 and 1.
- Decision Tree is used because my mini-thesis and paper publication is about building a credit scoring model using a decision tree algorithm, referring to Credit scoring in banks and financial institutions via data mining techniques: A literature review (Sadatrasoul, et al. 2013)


###1. Import

In [ ]:
#Libraries
!pip install scikit-plot #used for visualizing confusion matrix
import numpy as np
import pandas as pd
import csv
import scikitplot as skplt

from sklearn.linear_model import LogisticRegression #for Logistic Regression method
from sklearn import tree #for Decision Tree method
from sklearn.model_selection import train_test_split #train test split data
from sklearn.preprocessing import StandardScaler #data normalization on independent variable (X)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
#Dataset
from google.colab import files
uploaded = files.upload()
dfcredit = pd.read_excel("CreditScoringdataset.xlsx")

###2. Pre-processing Data

In [ ]:
#displays the top 5 rows of the dataset.
dfcredit.head()

In [26]:
#remove column "ID" because it is not a variable used in calculations/models
dfcredit = dfcredit.drop('ID',axis=1) #axis=1 (remove all values from the column)

In [ ]:
#view data info
dfcredit.info()

In [ ]:
#view the number of missing values
dfcredit.isna().sum() #isna() = view missing values, notna() = view filled values, sum() = count amount

In [ ]:
#fill in the missing value using the mean of each variable
meaninqtimelast = dfcredit['InqTimeLast'].mean()
meantlcnt = dfcredit['TLCnt'].mean()
meantlsum = dfcredit['TLSum'].mean()
meantlmaxsum = dfcredit['TLMaxSum'].mean()
meantlsatcnt = dfcredit['TLSatCnt'].mean()
meantl75utilcnt = dfcredit['TL75UtilCnt'].mean()
meantl50utilcnt = dfcredit['TL50UtilCnt'].mean()
meantlbalhcpct = dfcredit['TLBalHCPct'].mean()
meantlsatpct = dfcredit['TLSatPct'].mean()
meantlopenpct = dfcredit['TLOpenPct'].mean()
meantlopen24pct = dfcredit['TLOpen24Pct'].mean()

dfcredit['InqTimeLast'] = dfcredit['InqTimeLast'].fillna(meaninqtimelast)
dfcredit['TLCnt'] = dfcredit['TLCnt'].fillna(meantlcnt)
dfcredit['TLSum'] = dfcredit['TLSum'].fillna(meantlsum)
dfcredit['TLMaxSum'] = dfcredit['TLMaxSum'].fillna(meantlmaxsum)
dfcredit['TLSatCnt'] = dfcredit['TLSatCnt'].fillna(meantlsatcnt)
dfcredit['TL75UtilCnt'] = dfcredit['TL75UtilCnt'].fillna(meantl75utilcnt)
dfcredit['TL50UtilCnt'] = dfcredit['TL50UtilCnt'].fillna(meantl50utilcnt)
dfcredit['TLBalHCPct'] = dfcredit['TLBalHCPct'].fillna(meantlbalhcpct)
dfcredit['TLSatPct'] = dfcredit['TLSatPct'].fillna(meantlsatpct)
dfcredit['TLOpenPct'] = dfcredit['TLOpenPct'].fillna(meantlopenpct)
dfcredit['TLOpen24Pct'] = dfcredit['TLOpen24Pct'].fillna(meantlopen24pct)

dfcredit.isna().sum()

###3. Data Train & Data Test (Train Test Split)

In [30]:
#split the dataset into 2 parts, data train and data test, y = target/label

x = dfcredit.iloc[:, 1:29].values #variable x
y = dfcredit.iloc[:, 0].values #variable y (TARGET)

In [31]:
#split with 75% data train & 25% data test ratio

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0) #random state = to ensure that the model delivers consistent results every time it is run

###4. Data Normalization
Normalize the value of the independent variable (X) because it has a value with a different range, so it is normalized until the data is in the value from 0 to 1

In [32]:
datanorm = StandardScaler()
x_train = datanorm.fit_transform(x_train)
x_test = datanorm.transform(x_test)

###5. Classification Model
Algoritma:
- Logistic Regression
- Decision Tree

In [ ]:
#Logistic Regression
dflogreg =  LogisticRegression()
dflogreg.fit(x_train, y_train)

#Predict Data Test
dflogreg_y = dflogreg.predict(x_test)

#Confusion Matrix
cm_dflogreg = confusion_matrix(y_test, dflogreg_y)
cm_dflogreg

In [ ]:
#Confusion Matrix Linear Regression Visualization
skplt.metrics.plot_confusion_matrix(y_test, dflogreg_y)

In [ ]:
#Decision Tree
dftree =  tree.DecisionTreeClassifier(random_state=0)
dftree.fit(x_train, y_train)

#Predict Test Data
dftree_y = dftree.predict(x_test)

#Confusion Matrix
cm_dftree = confusion_matrix(y_test, dftree_y)
cm_dftree

In [ ]:
#Confusion Matrix Decision Tree Visualization
skplt.metrics.plot_confusion_matrix(y_test, dftree_y)

###Performansi Model Confusion Matrix

In [ ]:
#Logistic Regression
print(accuracy_score(y_test, dflogreg_y))

In [ ]:
#Decision Tree
print(accuracy_score(y_test, dftree_y))

The Logistic Regression algorithm has a higher accuracy value than Decision Tree, so the model will continue to use the algorithm.

In [ ]:
#Predict & Probability Value of Logistic Regression
logreg_pred = dflogreg.predict_proba(x_test)
logreg_pred

In [ ]:
#Combination Model of Probability, Predicted, and Actual Values

dflogregprob = pd.DataFrame(logreg_pred, columns = ['goodloan_prob', 'defaulters_prob'])
dflogregpred = pd.DataFrame(dflogreg.predict(x_test), columns = ['Predicted Value'])
dflogregactual = pd.DataFrame(y_test,columns= ['Actual Value'])

dfcreditprob=pd.concat([dflogregactual, dflogregprob, dflogregpred], axis=1)
dfcreditprob.to_csv('CreditScoring_Probability.csv')
dfcreditprob.head()